# *Facebook Prophet*

In [91]:
from fbprophet import Prophet
from fbprophet.plot import plot_components_plotly
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error
import plotly.graph_objects as go
from fbprophet.diagnostics import cross_validation, performance_metrics
import numpy as np
from sklearn.model_selection import TimeSeriesSplit


In [52]:
import warnings
warnings.filterwarnings("ignore")

The Facebook Prophet model has a number of advantages over the tried and tested SARIMA and exponential smoothing models. Perhaps most useful to this project is the ability of the Prophet model to account for multiple seasonalities. This means that we can use this model on much more granular data. We saw from the EDA that seasonality exists on multiple scales in our power consumption data. For this model, we will use the daily granularity which features both a weekly, and yearly seasonality.

The Prophet model can also account for changepoints in the trend. That is, the model can account for an inconsistent trend in the data. From our seasonal decomposition, we saw that the trend is not completely linear, which could be the source of some error in the SARIMA and exponential smoothing models. These changepoints can be specified manually, however since the change in trend is not particularly drastic, we will allow the model to account for this automatically.

Additionally, the Prophet model can also account for exogenous variables and holidays. These features could be used to account for weather conditions and bank holidays, which intuitively have an effect on power consumption. Whilst outside the current scope of this project, this is certainly an element which I would like to add.

The downside of the Prophet model is its lack of interpretability. The model does not provide any metrics to show which parameters it is considering in the predictions, and there is limited documentation on how the model works.

In [53]:
# Loading daily data
fb_Std = pd.read_pickle('data/LCL_unstack_day.pkl')
fb_Std.head()

tariff           Std       ToU
DateTime                      
2012-01-01  0.523669  0.528838
2012-01-02  0.534882  0.543422
2012-01-03  0.536094  0.554564
2012-01-04  0.537206  0.540528
2012-01-05  0.541646  0.533450

The Prophet model requires the input DataFrame to be in a specific format. The DateTime objects should be in a column called `ds`, whilst the actual values should be in a column labelled `y`.

In [54]:
# Put data into Prophet fromat

fb_df = fb_Std.reset_index(names = 'ds')    # Set ds column
fb_df.columns.name = ''                     # Remove 'tariff' index name

fb_ToU = fb_df.drop(columns='ToU')                      # Create ToU DataFrame
fb_ToU.rename(columns = {'Std':'y'}, inplace = True)    # Rename to 'y'

fb_ToU = fb_df.drop(columns='Std')                      # Create Std DataFrame
fb_ToU.rename(columns = {'ToU':'y'}, inplace = True)    # Rename to 'y'


---
## *Standard Tariff*

The `fbprophet` library has many proprietary functions. We will make use of the `cross_validation` and `performance_metrics` functions to carry out a cross validation on our model.

In [107]:
# Train test split 80:20
Std_train = fb_ToU.iloc[:round(0.8*len(fb_ToU))]
ToU_test = fb_ToU.iloc[Std_train.index[-1]+1:]

In [185]:
# Instantiate and train
Prophet_Std = Prophet(weekly_seasonality=True, yearly_seasonality=True, seasonality_mode='multiplicative')
Prophet_Std.fit(Std_train)

# Perform cross validation
# Initially train on 370 days to capture full year of seasonality (365.25 days)
# Forecast up to the end of Std_train. len(Std_train)-1 - 370 = horizon = 261 days
# Forecast windows of 30 days (period)
Std_cv_df = cross_validation(Prophet_Std, initial = '370 days', horizon = '261 days', period = '30 days')

# Score cross validation (MAPE)
cv_mape = round(np.mean(performance_metrics(Std_cv_df)['mape'])*100,2)

# Instantiate prediction dfs
Std_train_pred = Std_train[['ds']] 
Std_test_pred = ToU_test[['ds']]

# Predict train and test
Std_train_pred = Prophet_Std.predict(Std_train_pred)[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
Std_train_pred['y'] = Std_train['y'].values

Std_test_pred = Prophet_Std.predict(Std_test_pred)[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
Std_test_pred['y'] = ToU_test['y'].values

# Score train and test (MAPE)
train_mape = round((mean_absolute_percentage_error(Std_train_pred['y'], Std_train_pred['yhat']))*100,2)
train_mape_low = round((mean_absolute_percentage_error(Std_train_pred['y'], Std_train_pred['yhat_lower']))*100,2)
train_mape_up = round((mean_absolute_percentage_error(Std_train_pred['y'], Std_train_pred['yhat_upper']))*100,2)

test_mape = round((mean_absolute_percentage_error(Std_test_pred['y'], Std_test_pred['yhat']))*100,2)
test_mape_low = round((mean_absolute_percentage_error(Std_test_pred['y'], Std_test_pred['yhat_lower']))*100,2)
test_mape_up = round((mean_absolute_percentage_error(Std_test_pred['y'], Std_test_pred['yhat_upper']))*100,2) 

# Print outputs
print(f'=========================================================\
    \n======== Mean Absolute Percentage Errors (MAPEs) ========\
    \n=========================================================\
    \nCross-validated MAPE: {cv_mape}%\
    \n\nTrain MAPE: {train_mape}%\
    \nTrian MAPE (lower): {train_mape_low}%\
    \nTrain MAPE (upper): {train_mape_up}%\
    \n\nTest MAPE: {test_mape}%\
    \nTest MAPE (lower): {test_mape_low}%\
    \nTest MAPE (upper): {test_mape_up}%\
    \n========================================================')

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 1 forecasts with cutoffs between 2013-01-05 00:00:00 and 2013-01-05 00:00:00


  0%|          | 0/1 [00:00<?, ?it/s]

======== Mean Absolute Percentage Errors (MAPEs) ========    
Cross-validated MAPE: 8.73%    

Train MAPE: 3.78%    
Trian MAPE (lower): 7.76%    
Train MAPE (upper): 8.23%    

Test MAPE: 7.04%    
Test MAPE (lower): 4.53%    
Test MAPE (upper): 12.96%    


Running the Prophet model on standard tariff daily data produces better performance than the weekly predictions made by the SARIMA and exponential smoothing models in terms of cross validated, and test MAPEs.

Like the weekly models, the Prophet model overpredicts the test set, but since it also provides an upper and lower margin for the predictions, we can use the lower margin as our prediction which gives a test MAPE lower than both the SARIMA and exponential smoothing models.

The `fbprophet` library also has proprietary plotting functions. The `plot_components_plotly` function shows the contributions of the predicted trend and seasonalities.

In [186]:
fig = plot_components_plotly(Prophet_Std, Prophet_Std.predict(Std_test_pred[['ds']]))
fig.update_layout(title = 'FB Prophet Seasonal Components')
fig.show()

From this plot we can see that the model has picked up the downward trend. We can also see the predicted yearly and weekly trends showing increased power consumption in winter and on weekend days, as expected, when people tend to spend more time at home.

In [187]:
fig = go.Figure()
fig.add_trace(go.Line(x=Std_train.ds, y=Std_train_pred['y'], mode='lines', name="Train"))
fig.add_trace(go.Line(x=ToU_test.ds, y=Std_test_pred['y'], mode='lines', name="Test"))
fig.add_trace(go.Line(x=Std_train.ds, y=Std_train_pred['yhat'], mode='lines', name="Train Predictions"))
fig.add_trace(go.Line(x=ToU_test.ds, y=Std_test_pred['yhat_lower'], mode='lines', name="Test Predictions")) # Use most prediction with lowest error
fig.update_xaxes(title_text = 'Date-Time')
fig.update_yaxes(title_text = 'Power, KW')
fig.update_layout(title = 'FB Prophet Performance on Standard Tariff Data')
fig.show()

c:\Users\triha\anaconda3\envs\timeseries\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




The Prophet model performs very well on predicting the seasonalities. We can see from the plot that the weekend peaks line up accurately with the actual data. The model does however struggle to capture the magnitude of variance during periods of high variance. This can be seen particularly clearly in the winter months. Despite having a multiplicative seasonality specified, the model appears to predict similar variance at all points in the seasonal cycle.

---
## *Variable Tariff*

In [188]:
# Train test split 80:20
ToU_train = fb_ToU.iloc[:round(0.8*len(fb_ToU))]
ToU_test = fb_ToU.iloc[ToU_train.index[-1]+1:]

In [190]:
# Instantiate and train
Prophet_ToU = Prophet(weekly_seasonality=True, yearly_seasonality=True, seasonality_mode='multiplicative')
Prophet_ToU.fit(ToU_train)

# Perform cross validation
# Initially train on 370 days to capture full year of seasonality (365.25 days)
# Forecast up to the end of Std_train. len(Std_train)-1 - 370 = horizon = 261 days
# Forecast windows of 30 days (period)
ToU_cv_df = cross_validation(Prophet_ToU, initial = '370 days', horizon = '261 days', period = '30 days')

# Score cross validation (MAPE)
cv_mape = round(np.mean(performance_metrics(ToU_cv_df)['mape'])*100,2)

# Instantiate prediction dfs
ToU_train_pred = ToU_train[['ds']] 
ToU_test_pred = ToU_test[['ds']]

# Predict train and test
ToU_train_pred = Prophet_ToU.predict(ToU_train_pred)[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
ToU_train_pred['y'] = ToU_train['y'].values

ToU_test_pred = Prophet_ToU.predict(ToU_test_pred)[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
ToU_test_pred['y'] = ToU_test['y'].values

# Score train and test (MAPE)
train_mape = round((mean_absolute_percentage_error(ToU_train_pred['y'], ToU_train_pred['yhat']))*100,2)
train_mape_low = round((mean_absolute_percentage_error(ToU_train_pred['y'], ToU_train_pred['yhat_lower']))*100,2)
train_mape_up = round((mean_absolute_percentage_error(ToU_train_pred['y'], ToU_train_pred['yhat_upper']))*100,2)

test_mape = round((mean_absolute_percentage_error(ToU_test_pred['y'], ToU_test_pred['yhat']))*100,2)
test_mape_low = round((mean_absolute_percentage_error(ToU_test_pred['y'], ToU_test_pred['yhat_lower']))*100,2)
test_mape_up = round((mean_absolute_percentage_error(ToU_test_pred['y'], ToU_test_pred['yhat_upper']))*100,2) 

# Print outputs
print(f'=========================================================\
    \n======== Mean Absolute Percentage Errors (MAPEs) ========\
    \n=========================================================\
    \nCross-validated MAPE: {cv_mape}%\
    \n\nTrain MAPE: {train_mape}%\
    \nTrian MAPE (lower): {train_mape_low}%\
    \nTrain MAPE (upper): {train_mape_up}%\
    \n\nTest MAPE: {test_mape}%\
    \nTest MAPE (lower): {test_mape_low}%\
    \nTest MAPE (upper): {test_mape_up}%\
    \n========================================================')

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 1 forecasts with cutoffs between 2013-01-05 00:00:00 and 2013-01-05 00:00:00


  0%|          | 0/1 [00:00<?, ?it/s]

======== Mean Absolute Percentage Errors (MAPEs) ========    
Cross-validated MAPE: 13.52%    

Train MAPE: 3.87%    
Trian MAPE (lower): 7.68%    
Train MAPE (upper): 8.26%    

Test MAPE: 7.17%    
Test MAPE (lower): 4.15%    
Test MAPE (upper): 13.58%    


Interestingly, the Prophet model produces a slightly worse cross validated MAPE for the variable tariff data than the standard tariff data. We know that the Prophet model struggles to predict high variance periods, so it could be the case that the variable tariff data contains more variance.

In [191]:
fig = plot_components_plotly(Prophet_ToU, Prophet_ToU.predict(ToU_test_pred[['ds']]))
fig.update_layout(title = 'FB Prophet Seasonal Components')
fig.show()

In [192]:
fig = go.Figure()
fig.add_trace(go.Line(x=ToU_train.ds, y=ToU_train_pred['y'], mode='lines', name="Train"))
fig.add_trace(go.Line(x=ToU_test.ds, y=ToU_test_pred['y'], mode='lines', name="Test"))
fig.add_trace(go.Line(x=ToU_train.ds, y=ToU_train_pred['yhat'], mode='lines', name="Train Predictions"))
fig.add_trace(go.Line(x=ToU_test.ds, y=ToU_test_pred['yhat_lower'], mode='lines', name="Test Predictions")) # Use most prediction with lowest error
fig.update_xaxes(title_text = 'Date-Time')
fig.update_yaxes(title_text = 'Power, KW')
fig.update_layout(title = 'FB Prophet Performance on Standard Tariff Data')
fig.show()

c:\Users\triha\anaconda3\envs\timeseries\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




As expected, we see a higher amount of variance in the variable tariff data, particularly towards the beginning of the dataset. This increased variance is not present in the succeeding seasonal cycles. This is likely due to the variable tariff households taking time to get used to their tariff and gradually adjusting their behaviours. The high variance confirms the idea above that this is the source of the increased cross validation error. Moreover, this is likely also the cause of the consistently higher errors on the variable tariff predictions for the weekly models.